## category 1 : orange (top)

#### to-do
1. 향수 정보 열기
2. <카테고리  3> : top_notes에 'rose'를 포함하는 향수만 필터링
3. top_notes 리스트에의 향들 : 카테고리 3에 포함되는 향 갯수, 다음 가까운 카테고리에 포함되는 향 갯수, 그 다음...
4. 각 갯수 * 해당 가중치(1, 0.7, 0.4) ==> 'score'
5. score에 따라서 정렬 후 top 3 향수 추출
6. 저장


In [1]:
import pandas as pd
from ast import literal_eval

import warnings
warnings.filterwarnings("ignore")

### 데이터 불러오기

In [2]:
# 향수 데이터를 필요한 칼럼만 남겨 perfume_df에 불러오기
perfume_df = pd.read_csv('../data/1st treatment_perfume.csv', index_col = 0)
perfume_df = perfume_df[['pk', 'name', 'top_notes', 'heart_notes', 'base_notes']]

# perfume_df의 note들 정보가 리스트가 아닌 string으로 저장되어 있으므로 리스트로 변환
note_types = ['top_notes', 'heart_notes', 'base_notes']
for n_type in note_types:
    perfume_df[n_type] = perfume_df[n_type].apply(literal_eval)

# 각 카테고리별 향들의 정보가 들어있는 cate_note.csv불러오기
cat_notes = pd.read_csv('../data/cate_note.csv', index_col = 0)

# note에 대한 정보 (이름과  pk)을 가진 데이터 프레임
note1 = pd.read_csv('../data/concat_notes.csv')
note1 = note1[['pk', 'name']]
note1[note1['name'] == 'orange'] # orange pk값 확인

,pk,name
659,660,orange


In [10]:
a = note1[note1['name'] == 'orange']['pk']
a.values[0]

660

***

### 타겟 향이 top에 포함되는 향수 추출

#### 1. top 필터링

In [3]:
# top_note에 타겟 향 (660)을 포함하는 향수 pk값 리스트로 반환
top_660 = []

for i in range(len(perfume_df)):
    for j in range(len(perfume_df['top_notes'][i])):
        if perfume_df['top_notes'][i][j] == 660:
            top_660.append(perfume_df['pk'][i])
print('top_notes 필터 후 : ', len(top_660))

# 반환된 리스트 값을 pk로 가지는 향수 정보를 보여주는 데이터 프레임
test = []
for i in top_660:
    test.append(perfume_df[perfume_df['pk'] == i])

df = test[0]
for i in range(1,len(test)):
    df = df.append(test[i])
df_final =df.reset_index(drop = True)
print('필터링된 데이터 프레임 크기 : ',len(df))

top_notes 필터 후 :  1048
필터링된 데이터 프레임 크기 :  1048


***

In [46]:
final = df_final[df_final['pk'] == 26120008]
final

,pk,name,top_notes,heart_notes,base_notes
0,26120008,Citron Citron,"[515, 530, 660]","[82, 604]","[178, 193, 649]"


In [4]:
filter_pk = []
for i in range(len(df_final)):
    if len(df_final['heart_notes'][i]) != 0 or len(df_final['base_notes'][i]) != 0:
        filter_pk.append(df_final['pk'][i])
len(filter_pk)

775

In [5]:
filtered = df_final[df_final['pk'] == filter_pk[0]]
for i in range(1, len(filter_pk)):
    filtered = filtered.append(df_final[df_final['pk'] == filter_pk[i]])
filtered.head()

,pk,name,top_notes,heart_notes,base_notes
0,26120008,Citron Citron,"[515, 530, 660]","[82, 604]","[178, 193, 649]"
1,26120030,4711 Echt Kölnisch Wasser,"[96, 515, 639, 660, 715]","[785, 793]","[624, 959]"
2,26120166,Barbier des Isles,"[96, 515, 660]","[178, 237, 472, 473]","[624, 627, 692, 785, 924]"
3,26120202,Black Label,"[96, 515, 519, 660]","[229, 372, 512, 785]","[649, 692, 808, 924]"
4,26120245,Boucheron pour Homme,"[82, 96, 512, 515, 519, 562, 660]","[180, 461, 473, 529, 785, 966]","[26, 472, 624, 649, 808, 908, 924]"


#### 필터링이 된 df(df_final)에서 점수 계산 후 탑3개
* cat1 = 1, cat11 = 0.7, cat8 = 0.3 점

In [19]:
df_final = filtered.reset_index(drop = True)
df_final['pk'][5]

26120315

In [6]:
cat1 = cat_notes['note_idx'].apply(literal_eval)[1] # 1번 카테고리에 해당하는 note들의 pk값 리스트
cat11 = cat_notes['note_idx'].apply(literal_eval)[11] # 11번 카테고리에 해당하는 note들의 pk값 리스트
cat8 = cat_notes['note_idx'].apply(literal_eval)[8] # 8번 카테고리에 해당하는 note들의 pk값 리스트

In [20]:
# 타겟 카테고리에 해당하는 향을 카테고리에 따라 a 또는 b 또는 c로 변경하는 함수
def convert_note(target_type):
    for i in range(len(df_final)):
        for j in range(len(df_final[target_type][i])):
            if df_final[target_type][i][j] in cat1:
                df_final[target_type][i][j] = 'a'
            elif df_final[target_type][i][j] in cat11:
                df_final[target_type][i][j] = 'b'
            elif df_final[target_type][i][j] in cat8:
                df_final[target_type][i][j] = 'c'
            else:
                pass

In [22]:
# target_type을 *_note 열들로 지정하고 위 조건에 해당하는 값을 변경
target_type = ['top_notes', 'heart_notes', 'base_notes']
for i in target_type:
    convert_note(i)
    
df_final.head(3)

,pk,name,top_notes,heart_notes,base_notes
0,26120008,Citron Citron,"[a, a, a]","[82, 604]","[178, c, c]"
1,26120030,4711 Echt Kölnisch Wasser,"[a, a, a, a, a]","[785, 793]","[624, c]"
2,26120166,Barbier des Isles,"[a, a, a]","[178, 237, 472, 473]","[624, 627, c, 785, c]"


In [23]:
# Import Counter from collections
from collections import Counter

# 데이터 프레임에 score열을 만들고 위에서 번환된 note값에 따라 점수 계산
# 10번 카테고리 향은 1점, 6번 카테고리 향은 0.7점, 9번 카테고리 향은 0.4점

# 점수 계산 함수
def score_cal(target_type):
    score = Counter(df_final[target_type][i])['a'] * 1 + Counter(df_final[target_type][i])['b'] * 0.7 + Counter(df_final[target_type][i])['c'] * 0.4
    return score

df_final['score'] = 0
for i in range(len(df_final)):
    t_score = score_cal('top_notes')
    m_score = score_cal('heart_notes')
    b_score = score_cal('base_notes')
    df_final['score'][i] = t_score + m_score + b_score


In [24]:
# score값에 따른 top 3
df_final.sort_values(by = 'score', ascending = False)[0:3]

,pk,name,top_notes,heart_notes,base_notes,score
424,26143106,Grace,"[a, a, a, a, a, a, a]","[178, 229, 473, 512, a, 646, 708, 927]","[c, b, 920]",9.1
327,26137943,Orange Spice,"[a, a, a, a, a]","[224, 259, a, 661, a, 725]","[28, 191, 652, c, b, c]",8.5
723,26157731,Yuzu (Japanese citrus fruit),"[a, a, a, 896]","[a, a, a]","[a, 785]",7.0


In [25]:
# 해당 향수의 전체 노트의 수 = 'total_notes'
df_final['total_notes'] = None
# 해당 향수의 평균 점수
df_final['score_avg'] = 0

# 노트들의 리스트 길이를 이용하여 평균 점수 구하기
for i in range(len(df_final)):
    df_final['total_notes'][i] = len(df_final['top_notes'][i] + df_final['heart_notes'][i] + df_final['base_notes'][i])
    df_final['score_avg'][i] = df_final['score'][i]/df_final['total_notes'][i]


In [26]:
# 평균 점수로 정한 top 3향수
df_final.sort_values(by = 'score_avg', ascending = False)[0:3]

,pk,name,top_notes,heart_notes,base_notes,score,total_notes,score_avg
246,26134666,Eau Monumentale,"[a, a, a]",[a],[c],4.4,5,0.880000
157,26129750,Refreshing,"[a, a, a, a]",[a],[624],5.0,6,0.833333
95,26125004,Acqua di Genova Colonia,"[a, a, a]","[a, 785]",[],4.0,5,0.800000


In [36]:
df_base = pd.read_csv('../data/fill_season_catagory.csv')

# 점수 계산 후 나온 상위 3개의 향수의 pk값을 이용하여 향수 최종 데이터에서 동일 pk값을 가진 향수 정보를 추출 
result = df_final.sort_values(by = 'score', ascending = False)[0:3]
result_pk = result['pk'].reset_index(drop=True)

fin = df_base[df_base['pk'] == result_pk[0]]
for i in range(1, len(result_pk)):
    fin = fin.append(df_base[df_base['pk'] == result_pk[i]])
    
# 결과 데이터 프레임 csv로 저장
fin.to_csv('../data/category1_result.csv')